In [14]:
# data_cleaning.py
# Contains functions to detect and handle inconsistent data and outliers
# Input:
#     - data/educatec_data/merged_educatec_moodle.csv
# Output:
#     - data/working_data/cleaned_data.csv

import os
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

def load_data(file_path):
    """
    Load the csv file from file_path
    :param file_path: Path - path to the csv file
    :return: pd.DataFrame - loaded dataframe
    """
    return pd.read_csv(file_path)



Análisis Exploratorio de Datos (EDA)
Antes de realizar cualquier limpieza, es importante entender el estado actual de los datos. Realizamos un análisis exploratorio de datos para obtener información sobre el número de filas, columnas, tipos de datos y valores faltantes."

In [15]:
def eda(df, file_name):
    """
    Perform exploratory data analysis on the dataframe
    :param df: pd.DataFrame - dataframe to analyze
    :return: None
    """
    num_zeros = (df == '0').sum().sum() + (df == 0).sum().sum()
    num_na = df.isnull().sum().sum() + num_zeros

    print(f"Número de valores nulos (incluyendo '0' y 0): {num_na}")
    print(f"Número de 'Sin respuesta': {(df == 'Sin respuesta').sum().sum()}")
    print(f"Número de 'Sin fecha': {(df == 'Sin fecha').sum().sum()}")

Aseguramos la calidad de los datos limpiando los valores que tengan 'Sin respuesta', 'Sin fecha', o que sean valores nulos para después eliminar esos datos. 

In [16]:
def replace_missing_values(df):
    """
    Replace 'sin respuesta' and 'sin fecha' with NaN
    :param df: pd.DataFrame - dataframe to process
    :return: pd.DataFrame - dataframe with missing values replaced
    """
    return df.replace(['Sin respuesta', 'Sin fecha', '0', 0, ''], pd.NA)

def drop_invalid_rows(df, file_name):
    """
    Drop rows with any NaN values.
    :param df: pd.DataFrame - dataframe to process
    :param file_name: str - name of the file being processed
    :return: pd.DataFrame - dataframe with invalid rows dropped
    """
    print(f"Shape of {file_name} before dropping invalid rows: {df.shape}")
    df = df.dropna()
    print(f"Shape of {file_name} after dropping invalid rows: {df.shape}")
    return df

def detect_inconsistencies(df):
    """
    Detect and report inconsistencies in the dataset
    :param df: pd.DataFrame - dataframe to analyze
    :return: pd.DataFrame - dataframe with inconsistencies marked
    """
    for col in df.select_dtypes(include=[np.number]).columns:
        df[f'{col}_inconsistent'] = df[col] < 0
    return df

def impute_missing_values(df):
    """
    Impute missing values in the dataframe
    :param df: pd.DataFrame - dataframe to process
    :return: pd.DataFrame - dataframe with missing values imputed
    """
    # Set the option to opt-in to the future behavior for fillna
    pd.set_option('future.no_silent_downcasting', True)

    # Impute numerical columns with mean
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

    # Impute categorical columns with mode
    cat_cols = df.select_dtypes(exclude=[np.number]).columns
    for col in cat_cols:
        mode = df[col].mode()[0] if not df[col].mode().empty else pd.NA
        df[col] = df[col].fillna(mode)

    return df


def detect_outliers(df):
    """
    Detect and report outliers in the dataset
    :param df: pd.DataFrame - dataframe to analyze
    :return: pd.DataFrame - dataframe with outliers marked
    """
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[f'{col}_outlier'] = (df[col] < lower_bound) | (df[col] > upper_bound)
    return df

def cleaning(input_data_path, output_data_path):
    """
    Perform data cleaning on the dataset
    :param input_data_path: Path - path to the input dataset
    :param output_data_path: Path - path to save the cleaned dataset
    :return: None
    """
    file_name = os.path.basename(input_data_path)
    dataset = load_data(input_data_path)
    print(f"Resumen del archivo antes de la limpieza: {file_name}")
    eda(dataset, file_name)  # EDA antes de la limpieza\n",
    dataset = replace_missing_values(dataset)
    dataset = drop_invalid_rows(dataset, file_name)
    dataset = detect_inconsistencies(dataset)
    dataset = impute_missing_values(dataset)
    dataset = detect_outliers(dataset)
    print(f"Resumen del archivo después de la limpieza: {file_name}")
    eda(dataset, file_name)  # EDA después de la limpieza\n",
    dataset.to_csv(output_data_path, index=False)

In [17]:
# Configurar rutas relativas para los directorios de datos
educatec_directory = os.path.join(os.getcwd(), "..", "..", "data", "educatec_data")
moodle_directory = os.path.join(os.getcwd(), "..", "..", "data", "moodle_data")
working_directory = os.path.join(os.getcwd(), "..", "..", "data", "working_data")

# Convertir a objetos Path y resolver las rutas
educatec_directory_path = Path(educatec_directory).resolve()
moodle_directory_path = Path(moodle_directory).resolve()
working_directory_path = Path(working_directory).resolve()

# Imprimir las rutas para verificar
print("Educatec Directory:", educatec_directory_path)
print("Moodle Directory:", moodle_directory_path)
print("Working Directory:", working_directory_path)


Educatec Directory: /Users/administrador/Downloads/Leaders4Edu/data/educatec_data
Moodle Directory: /Users/administrador/Downloads/Leaders4Edu/data/moodle_data
Working Directory: /Users/administrador/Downloads/Leaders4Edu/data/working_data


In [18]:
def main():
    """
    Main function
    :return: None
    """

    # Paths para los diferentes datasets
    input_data_path = educatec_directory_path / "educatec.csv"
    output_data_path = working_directory_path / "cleaned_data_educatec.csv"
    cleaning(input_data_path, output_data_path)
    
    input_data_path = moodle_directory_path / "course_modules_completion.csv"
    output_data_path = working_directory_path / "cleaned_data_course_modules_completion.csv"
    cleaning(input_data_path, output_data_path)

    input_data_path = moodle_directory_path / "course_modules.csv"
    output_data_path = working_directory_path / "cleaned_data_course_modules.csv"
    cleaning(input_data_path, output_data_path)

    input_data_path = moodle_directory_path / "user_info_data.csv"
    output_data_path = working_directory_path / "cleaned_data_user_info_data.csv"
    cleaning(input_data_path, output_data_path)

    input_data_path = moodle_directory_path / "users.csv"
    output_data_path = working_directory_path / "cleaned_data_users.csv"
    cleaning(input_data_path, output_data_path)

if __name__ == "__main__":
    main()


Resumen del archivo antes de la limpieza: educatec.csv
Número de valores nulos (incluyendo '0' y 0): 0
Número de 'Sin respuesta': 202212
Número de 'Sin fecha': 202212
Shape of educatec.csv before dropping invalid rows: (205260, 7)
Shape of educatec.csv after dropping invalid rows: (3048, 7)
Resumen del archivo después de la limpieza: educatec.csv
Número de valores nulos (incluyendo '0' y 0): 6096
Número de 'Sin respuesta': 0
Número de 'Sin fecha': 0
Resumen del archivo antes de la limpieza: course_modules_completion.csv
Número de valores nulos (incluyendo '0' y 0): 833037
Número de 'Sin respuesta': 0
Número de 'Sin fecha': 0
Shape of course_modules_completion.csv before dropping invalid rows: (4710592, 6)
Shape of course_modules_completion.csv after dropping invalid rows: (3878191, 6)
Resumen del archivo después de la limpieza: course_modules_completion.csv
Número de valores nulos (incluyendo '0' y 0): 23263990
Número de 'Sin respuesta': 0
Número de 'Sin fecha': 0
Resumen del archivo a